### 单轮对话

In [ ]:
from openai import OpenAI
client = OpenAI(
    base_url="https://open.bigmodel.cn/api/paas/v4",
    api_key="77ff713fcbaf89e4dc2e623868e7e829.IUDHIW1kf1UdMB0f"
)
completion = client.chat.completions.create(
    model='glm-4-flash',
    messages=[
        {'role': 'system', 'content': "你是一个AI小助手，你叫小智，你可以帮助用户处理表格。"},
        {'role': 'user', 'content': "你叫什么名字？你可以做什么？"}
    ],
    max_tokens=1024,
    temperature=0.7,
)
completion.choices[0].message.content

### LangChain单轮对话

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你好，我是你的AI小助手————{name}。"),
    ("human", "你好，你叫什么名字？你可以做什么？"),
])
prompt_template.format(name="小智")

In [10]:
model = ChatOpenAI(
    api_key="77ff713fcbaf89e4dc2e623868e7e829.IUDHIW1kf1UdMB0f",
    base_url="https://open.bigmodel.cn/api/paas/v4",
    model="glm-4-flash",
    max_tokens=1024,
    temperature=0.7,
)

In [34]:
def output_parser(output: str):
    parser_model = ChatOpenAI(
        api_key="ff8ea6bbf49622f361baa8dcb0e6103e.564HDzPUS8rNujSw",
        base_url="https://open.bigmodel.cn/api/paas/v4",
        model="glm-3-turbo",
        max_tokens=1024,
        temperature=0.3,
    )
    prompt = "你需要将传入的文本改写，尽可能更自然。这是你需要改写的文本:'{text}'"
    return parser_model.invoke(prompt.format(text=output))

In [ ]:
chain = prompt_template | model | output_parser
answer = chain.invoke({"name": "小智"})
print(answer.content)

### 多轮对话

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你好，我是你的AI小助手————小杰。"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{query}"),
])
prompt_template.invoke({
    "name": "小智",
    "chat_history": [
        ("human", "讲一个故事"),
        ("ai", "龟兔赛跑"),
    ],
    "query": "讲一个笑话",
}).messages

In [ ]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(
    api_key="77ff713fcbaf89e4dc2e623868e7e829.IUDHIW1kf1UdMB0f",
    base_url="https://open.bigmodel.cn/api/paas/v4",
    model="glm-4-flash",
    max_tokens=1024,
    temperature=0.7,
)
chain = prompt_template | model
chat_history = []

def chat(query):
    answer = chain.invoke({"name": "小智", "chat_history": chat_history, "query": query})
    chat_history.extend([("human", query), ("ai", answer.content)])
    return answer.content
chat("你叫什么名字？")

### LangChain多轮对话

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
chat_history = ChatMessageHistory()
chat_history.add_ai_message("你好，我是你的AI小助手————")
chat_history.add_user_message("你好，我是一名算法工程师，我今年22岁了")
chat_history.add_ai_message("你好，我是你的AI小助手————")
chat_history.messages

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory
chain = prompt_template | model
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda x: chat_history,
    input_messages_key="query",
    history_messages_key="chat_history",
)
chain_with_history.invoke(
    input={"query": "我的职业是什么？你叫什么？"},
    config={"configurable": {"session_id": "unused"}},
).content
chat_history.messages

In [ ]:
def trim_history(chain_input):
    messages = chat_history.messages
    if len(messages) > 2:
        chat_history.clear()
        for message in messages[-2:]:
            chat_history.add_message(message)
    return chain_input
chain_with_trimming = trim_history | chain_with_memory
chain_with_trimming.invoke(
    input={"query": "你叫什么？"},
    config={"configurable": {"session_id": "unused"}},
).content
chat_history.messages

In [27]:
def summarize_messages(chain_input):
    messages = chat_history.messages
    if len(messages)>=6:
        summarization_prompt = ChatPromptTemplate.from_messages([
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "将上述聊天信息提炼为一条摘要信息，尽可能多地包含具体细节。")
        ])
        summarize_chain = summarization_prompt | model
        summary_message = summarize_chain.invoke({"chat_history": messages})
        chat_history.clear()
        chat_history.add_message(summary_message)
    return chain_input
chain_with_summary = summarize_messages | chain_with_memory

In [ ]:
chat_history.clear()
chat_history.add_ai_message("你好，我是你的AI小助手————小智")
chat_history.add_user_message("你好，我是一名算法工程师，我今年22岁了")
chat_history.add_ai_message("你好，有什么我能帮助的吗")
chat_history.add_user_message("现在我想要学习深度学习，请问你推荐什么书籍")
chat_history.add_ai_message("深度学习入门书籍推荐：\n1.深度学习入门指南\n2.深度学习入门\n3.深度学习入门（Python版）")
chat_history.add_user_message("谢谢")
chat_history.messages

In [ ]:
chain_with_summary.invoke(
    input={"query": "我的职业是什么？"},
    config={"configurable": {"session_id": "unused"}},
).content
chat_history.messages

### Agent

In [1]:
import pandas as pd
from pandasql import sqldf
from langchain.tools import tool

@tool
def simulate_database_operation(sql: str):
    '''根据sql语句操作数据库中的表'''
    df_table = pd.DataFrame({
        'time': ['2024-09-10', '2024-09-11', '2024-09-12'], 
        'inin': [7, 8, 9], 
        'out': [4, 5, 6], 
        'total': [3, 3, 3]})
    result = sqldf(sql)
    return result
# simulate_database_operation('SELECT * FROM df_table')

In [2]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(
    api_key="77ff713fcbaf89e4dc2e623868e7e829.IUDHIW1kf1UdMB0f",
    base_url="https://open.bigmodel.cn/api/paas/v4",
    model="glm-4-flash",
    max_tokens=1024,
    temperature=0.7,
)

In [3]:
from langchain_community.tools import DuckDuckGoSearchRun
tools = [DuckDuckGoSearchRun(), simulate_database_operation]
# model_with_tools = model.bind_tools(tools)
# model_with_tools.invoke("你好")

In [ ]:
def call_tool(model_output: str, tools: list[tool]):
    tools_map = {tool.name.lower(): tool for tool in tools}
    tools_response = {}
    for tool in model_output.tool_calls:
        tool_name = tool['name']
        tool_args = tool['args']
        tool_instance = tools_map[tool_name]
        tool_response = tool_instance.invoke(*tool_args.values())
        tools_response[tool_name] = tool_response
    return tools_response

def manual_agent(query: str, model: ChatOpenAI, tools: list[tool]):
    model_with_tools = model.bind_tools(tools)
    model_output = model_with_tools.invoke(query)
    tool_response = call_tool(model_output, tools)
    final_response = model.invoke(f'original query: {query} \n\n\n tool response: {tool_response}')
    return final_response
manual_agent("查询df_table表中inin大于2的数据", model, tools).content
# manual_agent("插入df_table表中一行数据，time为2024-10-07，inin为4，out为4，total为0", model, tools).content

### Agent Langgraph

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
agent = create_react_agent(model, tools)
answer = agent.invoke({'messages': [HumanMessage("查询df_table表中out大于5的数据个数")]})
answer['messages'][-1].content

In [ ]:
from langchain.tools
from langchain_community.tools
from langchain_experimental.agents import create_pandas_dataframe_agent, create_csv_agent